#### Pydantic Basics: Creating and Using Models
Pydantic models are the foundation of data validation in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples.



In [1]:
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str
    

In [3]:
person=Person(name='judi',age=22,city='bangalore')
print(person)

Person(name='judi', age=22, city='bangalore')


In [11]:
#pydantic
from pydantic import BaseModel
class Person1(BaseModel):
    name:str
    age:int
    city:str


In [ ]:
person=Person1(name=11,age=22,city='bangalore')
person


ValidationError: 1 validation error for Person1
name
  Input should be a valid string [type=string_type, input_value=11, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [16]:
person=Person1(name='judi',age='22',city='bangalore')
person


Person1(name='judi', age=22, city='bangalore')

1.Automatic Type Validation and Conversion:

Pydantic automatically converts the age='22' string to an integer (22), because age is annotated as an int.

If the conversion fails (e.g., age='twenty-two'), it raises a ValidationError.

2.Data is Strictly Parsed:

Pydantic ensures all the fields follow the types you declared.

You’ll get helpful error messages when data doesn’t match the expected schema

In [17]:
from typing import Optional
class Employee(BaseModel):
    id:int
    name:str
    department:str
    salary:Optional[float]=None #optional with default value
    is_active: Optional[bool]=True

In [ ]:
emp1=Employee(id=1,name="judi",department='cs')
emp1



Employee(id=1, name='judi', department='cs', salary=50000.0, is_active=True)

In [20]:
emp2=Employee(id=1,name="judi",department='cs',salary='50000')
emp2

Employee(id=1, name='judi', department='cs', salary=50000.0, is_active=True)

Definition:
- Optional[type]: Indicates the field can be None

- Default value (= None or = True): Makes the field optional

- Required fields must still be provided

- Pydantic validates types even for optional fields when values are provided

In [21]:
from typing import List
class Classroom(BaseModel):
    room_no:str
    students: List[str] #list of strings
    capacity: int

In [24]:
classroom=Classroom(room_no="a1",students=('judi','rispah'),capacity=2)
classroom

Classroom(room_no='a1', students=['judi', 'rispah'], capacity=2)

In [25]:
classroom=Classroom(room_no="a1",students=('judi',12,'rispah'),capacity=2)
classroom

ValidationError: 1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [27]:
try:
    invalid_val=Classroom(room_no="A1",students=["judi",123],capacity=30)

except Exception as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


model with nested model

In [8]:
from pydantic import BaseModel

class Address(BaseModel):
    street:str
    city:str
    zip_code:str

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address #nested model
    


In [13]:
customer=Customer(customer_id=1,name='judi',
                  address={'street':'thomas','city':'tvl','zip_code':'123'})

In [15]:
customer

Customer(customer_id=1, name='judi', address=Address(street='thomas', city='tvl', zip_code='123'))

pydantic fields :customization and constraints

In [18]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str=Field(max_length=10,min_length=4,)
    price:float=Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int=Field(ge=0)

item=Item(name="Book", price=100000,quantity=10)
print(item)


ValidationError: 1 validation error for Item
price
  Input should be less than or equal to 10000 [type=less_than_equal, input_value=100000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

In [20]:
from pydantic import BaseModel,Field

class Item(BaseModel):
    name:str=Field(max_length=10,min_length=4,)
    price:float=Field(gt=0,le=10000)  ## greater than 0 and less than or equal to 1000
    quantity:int=Field(ge=0)

item=Item(name="Book", price=10000,quantity=10)
print(item)


name='Book' price=10000.0 quantity=10


In [ ]:
class User(BaseModel):
    username:str=Field(description="Unique username for the user")
    age:int=Field(default=18,description="User age default to 18 ")
    email:str= Field(default_factory=lambda: "user@example.com",description="Default email address")
#default_factory expects a callable (i.e., a function) that returns the default value

# Examples
user1 = User(username="alice")
print(user1)




username='alice' age=18 email='user@example.com'


In [22]:
user2 = User(username="bob", age=25, email="bob@domain.com")
print(user2)

username='bob' age=25 email='bob@domain.com'


In [23]:
User.model_json_schema()

{'properties': {'username': {'description': 'Unique username for the user',
   'title': 'Username',
   'type': 'string'},
  'age': {'default': 18,
   'description': 'User age default to 18 ',
   'title': 'Age',
   'type': 'integer'},
  'email': {'description': 'Default email address',
   'title': 'Email',
   'type': 'string'}},
 'required': ['username'],
 'title': 'User',
 'type': 'object'}